In [1]:
import pandas as pd
library=pd.read_csv('libraries.csv', low_memory=False)
states=pd.read_csv('states.csv', low_memory=False)

In [2]:
library.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9242 entries, 0 to 9241
Data columns (total 74 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   State                                  9242 non-null   object 
 1   Library ID                             9242 non-null   object 
 2   Submission Year                        9242 non-null   object 
 3   Library Name                           9242 non-null   object 
 4   Street Address                         9242 non-null   object 
 5   City                                   9242 non-null   object 
 6   Zip Code                               9242 non-null   int64  
 7   Longitude                              9242 non-null   float64
 8   Latitude                               9242 non-null   float64
 9   State Code                             9242 non-null   int64  
 10  County Code                            9242 non-null   int64  
 11  Coun

In [3]:
# Chuyển các cột sang dạng số (float hoặc int)
numeric_columns = [
    'Zip Code', 'Longitude', 'Latitude', 'State Code', 'County Code', 'County Population', 
    'Service Population', 'Service Population Without Duplicates', 'Total Staff', 
    'Total Operating Revenue', 'Total Operating Expenditures', 'Total Capital Revenue', 
    'Total Capital Expenditures', 'Print Collection', 'Digital Collection', 
    'Library Visits', 'Registered Users', 'Circulation Transactions'
]
for col in numeric_columns:
    library[col] = pd.to_numeric(library[col], errors='coerce')


In [4]:
# Danh sách các cột cần giữ lại
columns_to_keep = [
    'State', 'Library Name', 'City', 'County', 'Service Population', 'Total Staff',
    'Print Collection', 'Digital Collection', 'Library Visits', 'Registered Users',
    'Circulation Transactions', 'Start Date', 'End Date'
]

# Loại bỏ các cột không cần thiết
library = library[columns_to_keep]


In [5]:
# Kiểm tra dữ liệu thiếu
missing_data = library.isnull().sum()

# Điền giá trị 0 cho các cột số liệu bị thiếu
library.fillna(0, inplace=True)


In [6]:
# Loại bỏ các hàng trùng lặp
library.drop_duplicates(inplace=True)


In [7]:
# Chuẩn hóa tên cột
library.columns = library.columns.str.strip().str.lower().str.replace(' ', '_')


In [8]:
# Chuyển đổi cột ngày tháng
library['start_date'] = pd.to_datetime(library['start_date'], errors='coerce')
library['end_date'] = pd.to_datetime(library['end_date'], errors='coerce')


In [9]:
# Phân loại thư viện dựa trên số lượng sách
def classify_library(row):
    if row['print_collection'] > 100000:
        return 'Large'
    elif row['print_collection'] > 50000:
        return 'Medium'
    else:
        return 'Small'

library['library_size'] = library.apply(classify_library, axis=1)


In [10]:
# Tính toán tổng số ngày hoạt động
library['days_open'] = (library['end_date'] - library['start_date']).dt.days


In [11]:
# Tạo báo cáo đơn giản
report = library.groupby('state').agg({
    'print_collection': 'sum',
    'library_visits': 'sum',
    'registered_users': 'sum',
    'circulation_transactions': 'sum'
}).reset_index()
report

,state,print_collection,library_visits,registered_users,circulation_transactions
0,AK,2434236.0,3491307.0,358089.0,4792662.0
1,AL,9442083.0,17217399.0,2663713.0,20526318.0
2,AR,6304377.0,10973629.0,1615238.0,14390348.0
3,AZ,8229847.0,27609708.0,3118822.0,43672064.0
4,CA,65409430.0,164300175.0,21723648.0,222788583.0
5,CO,10316703.0,32981666.0,3588616.0,64683932.0
6,CT,14577630.0,21972583.0,1647190.0,31081616.0
7,DC,1815540.0,4230790.0,359371.0,3938767.0
8,DE,1642715.0,3834672.0,379791.0,6180769.0
9,FL,31602986.0,75553933.0,10615421.0,116693486.0


In [127]:
library_cleaned= library[library['service_population'] > 0]


In [135]:
library_cleaned

,state,library_name,city,county,service_population,total_staff,print_collection,digital_collection,library_visits,registered_users,circulation_transactions,start_date,end_date,library_size,days_open
0,AK,ANCHOR POINT PUBLIC LIBRARY,ANCHOR POINT,KENAI PENINSULA,2059.0,0.68,16843.0,0.0,5379.0,396.0,11607.0,2013-07-01,2014-06-30,Small,364.0
1,AK,ANCHORAGE PUBLIC LIBRARY,ANCHORAGE,ANCHORAGE,300549.0,78.90,481861.0,9773.0,942873.0,119619.0,1886620.0,2013-01-01,2013-12-31,Large,364.0
2,AK,ANDERSON VILLAGE LIBRARY,ANDERSON,DENALI,209.0,0.50,15304.0,0.0,874.0,161.0,1092.0,2013-07-01,2014-06-30,Small,364.0
3,AK,KUSKOKWIM CONSORTIUM LIBRARY,BETHEL,BETHEL,6241.0,3.00,34115.0,165.0,44980.0,1500.0,12347.0,2013-07-01,2014-06-30,Small,364.0
4,AK,BIG LAKE PUBLIC LIBRARY,BIG LAKE,MATANUSKA-SUSITNA,12793.0,3.00,23810.0,9773.0,43285.0,3878.0,59630.0,2013-07-01,2014-06-30,Small,364.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9237,WY,SUBLETTE COUNTY LIBRARY,PINEDALE,SUBLETTE,10041.0,15.00,73390.0,7766.0,197314.0,6073.0,119741.0,2013-07-01,2014-06-30,Medium,364.0
9238,WY,TETON COUNTY LIBRARY,JACKSON,TETON,22268.0,41.36,81207.0,7832.0,280211.0,32446.0,354163.0,2013-07-01,2014-06-30,Medium,364.0
9239,WY,UINTA COUNTY LIBRARY,EVANSTON,UINTA,21066.0,13.00,111377.0,7087.0,112403.0,16279.0,179197.0,2013-07-01,2014-06-30,Large,364.0
9240,WY,WASHAKIE COUNTY LIBRARY,WORLAND,WASHAKIE,8463.0,6.30,73464.0,7087.0,73300.0,6774.0,70396.0,2013-07-01,2014-06-30,Medium,364.0


In [26]:
states.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 62 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   Submission Year                        51 non-null     int64  
 1   State                                  51 non-null     object 
 2   State Code                             51 non-null     int64  
 3   Region Code                            51 non-null     int64  
 4   Service Population                     51 non-null     int64  
 5   Service Population Without Duplicates  51 non-null     int64  
 6   State Population                       51 non-null     int64  
 7   Central Libraries                      51 non-null     int64  
 8   Branch Libraries                       51 non-null     int64  
 9   Bookmobiles                            51 non-null     int64  
 10  MLS Librarians                         51 non-null     float64
 11  Libraria

In [42]:
# Kiểm tra giá trị thiếu
missing_values = states.isnull().sum()

# Thay thế giá trị thiếu bằng 0 nếu có
states.fillna(0, inplace=True)

# Chuyển đổi các cột ngày tháng từ object sang datetime
states['Start Date'] = pd.to_datetime(states['Start Date'])
states['End Date'] = pd.to_datetime(states['End Date'])

# Xóa các cột không cần thiết
columns_to_drop = [
    'Submission Year', 'State Code', 'Region Code', 'State Population', 
    'Service Population Without Duplicates', 'Local Cooperative Agreements', 
    'State Licensed Databases', 'Total Licensed Databases', 'Start Date', 
    'End Date', 'Local Government Capital Revenue', 'State Government Capital Revenue', 
    'Federal Government Capital Revenue', 'Other Capital Revenue', 
    'Other Operating Expenditures', 'Other Collection Expenditures', 'Print Subscriptions'
]
states_cleaned = states.drop(columns=columns_to_drop)

# Chuyển đổi các cột số liệu sang kiểu numeric nếu cần
numeric_columns = ['Service Population', 'Local Government Operating Revenue', 
                   'Total Staff Expenditures', 'Total Collection Expenditures', 'Print Collection']
states_cleaned[numeric_columns] = states_cleaned[numeric_columns].apply(pd.to_numeric, errors='coerce')

# Tạo các cột bổ sung: số lượng sách còn lại và tỷ lệ mượn sách
states_cleaned['Books Available'] = states_cleaned['Print Collection'] - states_cleaned['Circulation Transactions']
states_cleaned['Borrow Rate'] = states_cleaned['Circulation Transactions'] / states_cleaned['Print Collection']
# Chuẩn hóa tên cột
states_cleaned.columns = states_cleaned.columns.str.strip().str.lower().str.replace(' ', '_')
states_cleaned

,state,service_population,central_libraries,branch_libraries,bookmobiles,mls_librarians,librarians,employees,total_staff,local_government_operating_revenue,...,children’s_programs,young_adult_programs,library_program_audience,children’s_program_audience,young_adult_program_audience,public_internet_computers,internet_computer_use,wireless_internet_sessions,books_available,borrow_rate
0,AK,652274,85,16,1,58.81,112.53,190.51,303.04,32474960,...,8802,1528,279080,202322,25902,998,772129,479245,-2358426,1.968857
1,AL,4822023,221,76,14,329.58,736.22,1080.88,1817.10,88056291,...,21685,4844,1053347,696970,77371,5302,4359414,15224387,-11084235,2.173918
2,AR,2833849,54,179,2,142.01,286.40,767.09,1053.49,64665467,...,21081,4620,997901,712783,88721,2596,1970448,160450,-8085971,2.282596
3,AZ,10501253,85,135,11,461.32,546.24,1485.28,2031.52,157746903,...,37852,6702,1472963,1034731,109545,5593,8255038,5880952,-35442217,5.306545
4,CA,38322887,167,950,53,2976.50,3209.71,8388.34,11598.05,1265099905,...,206627,33645,9491467,6909344,531379,21735,35000501,14252610,-157379153,3.406062
5,CO,5238320,97,162,13,663.22,910.62,2340.11,3250.73,265575219,...,67253,9064,2647097,1799469,151930,6407,7395748,6234884,-54367229,6.269826
6,CT,4374214,192,47,4,752.51,1074.91,1210.12,2285.03,168823962,...,54737,7381,2003864,1226174,112501,4355,4465464,524318,-16503986,2.132145
7,DC,658893,1,25,0,147.00,147.00,423.00,570.00,53095222,...,8514,1566,284969,225815,15624,1000,1050623,-1,-2123227,2.169474
8,DE,925244,19,13,2,63.58,121.72,208.31,330.03,19805621,...,5408,1197,256274,175335,17349,772,622515,139907,-4538054,3.762533
9,FL,19813366,63,468,24,1669.69,1917.91,4074.37,5992.28,461436699,...,96748,17006,4611959,2928059,314348,16781,19076151,3276803,-85090500,3.692483


In [140]:
print(library_cleaned.info())
print(states_cleaned.info())

<class 'pandas.core.frame.DataFrame'>
Index: 9232 entries, 0 to 9241
Data columns (total 15 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   state                     9232 non-null   object        
 1   library_name              9232 non-null   object        
 2   city                      9232 non-null   object        
 3   county                    9232 non-null   object        
 4   service_population        9232 non-null   float64       
 5   total_staff               9232 non-null   float64       
 6   print_collection          9232 non-null   float64       
 7   digital_collection        9232 non-null   float64       
 8   library_visits            9232 non-null   float64       
 9   registered_users          9232 non-null   float64       
 10  circulation_transactions  9232 non-null   float64       
 11  start_date                9232 non-null   datetime64[ns]
 12  end_date                 

In [133]:
library_cleaned.to_csv('libraries_cleaned.csv', index=False)
states_cleaned.to_csv('states_cleaned.csv', index=False)